## Feature Engineering (Continued from `EDA_1`)
**Import required packages & check working directory**

In [15]:
import datetime
from EDA_fun_graph import ts_decompose
from EDA_fun_multithread import fit_linear_model, kendall_rank
import multiprocessing
from multiprocessing.pool import Pool
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KernelDensity
from statsmodels.nonparametric.kernel_density import KDEMultivariate
from statsmodels.tsa.stattools import adfuller
import time

if os.getcwd()[-3:] == "src":
    os.chdir(os.path.dirname(os.getcwd()))
else:
    pass


**User defined parameters**


In [16]:
ticker = "AAPL"
options_path = f"data/EDA1/{ticker}"
adj_daily_closing_path = "data/adjusted_daily_closing/"
dividends_path = "data/dividends/"

## Load Data

In [3]:
# Bookkeeping variables
COMPLETE_DICT = dict()
INCOMPLETE_DICT = dict()

for year in os.listdir(options_path):
    for file in os.listdir(os.path.join(options_path, year)):
        # Load
        temp_df = pd.read_csv(os.path.join(os.path.join(options_path, year, file)))

        # Convert columns to correct format
        temp_df["date"] = pd.to_datetime(temp_df["date"]).dt.date
        temp_df["expiration date"] = pd.to_datetime(temp_df["expiration date"]).dt.date

        if file.split("_")[-1] == "complete.csv":
            COMPLETE_DICT[year] = temp_df
        elif file.split("_")[-1] == "incomplete.csv":
            INCOMPLETE_DICT[year] = temp_df
        else:
            raise Exception("No other type of file should exist!")


In [17]:
date_close_df = pd.read_csv(os.path.abspath(os.path.join(adj_daily_closing_path, (ticker + ".csv"))))
date_close_df["date"] = pd.to_datetime(date_close_df["date"]).dt.date

dividends_df = pd.read_csv(os.path.abspath(os.path.join(dividends_path, (ticker + "_ts.csv"))))
dividends_df["date"] = pd.to_datetime(dividends_df["date"]).dt.date

date_close_df = date_close_df[["date", "close"]].merge(right=dividends_df,
                                                       how="inner", on="date")

date_close_df["adj close"] = (date_close_df["close"] - date_close_df["dividend"]).round(6)
date_close_df = date_close_df[["date", "adj close"]]


## Cells below to explore methods of preprocessing

Dickey-Fuller test on adjusted close price (segmented annually)

In [8]:
dickey_df = pd.DataFrame(columns=["year", "ADF statistic", "p-value"])

min_year = 2005
max_year = 2021

for n in range(min_year, max_year + 1):
    year_df = date_close_df[pd.to_datetime(date_close_df["date"]).dt.year == n]
    adf_year = adfuller(year_df["adj close"], regression="c", autolag="AIC")
    dickey_df = dickey_df.append({"year": n, "ADF statistic": adf_year[0], "p-value": adf_year[1]}, ignore_index=True)

dickey_df["year"] = dickey_df["year"].astype(int)

dickey_df

,year,ADF statistic,p-value
0,2005,0.252304,0.975047
1,2006,-0.991645,0.756299
2,2007,0.423419,0.982350
3,2008,-0.945886,0.772486
4,2009,-0.567357,0.878189
5,2010,-0.522971,0.887455
6,2011,-1.769742,0.395584
7,2012,-2.396842,0.142630
8,2013,-0.728815,0.839144
9,2014,-0.447082,0.901936


Since for most/all years, the p-value is greater than 0.05, we cannot reject the null hypothesis that a unit root is present.

Graph of time series, autocorrelation (ACF) and partial ACF of entire ticker history to pick appropriate de-trending options.

In [18]:
acf_df = date_close_df.copy()

acf_lags = []

# Close dates, do not include 0
acf_lags.extend(np.arange(1, 6))

# Month
acf_lags.extend(np.arange(15, 26))

# Quarter
acf_lags.extend(np.arange(75, 96))

# Year
acf_lags.extend(np.arange(240, 266))

acf_plot = ts_decompose(ts_df=acf_df, min_year=min_year, max_year=max_year,
                        lags=acf_lags, pacf_lag=10)

acf_plot.write_image("./img/EDA2_ACF.svg", width=1000, height=800)

![ACF](../img/EDA2_ACF.svg)

First take a differencing of 1 day as it is the overarching signal here, then re-examine for finer seasonality.

In [11]:
n = 1

acf_df[f"delta {n}"] = acf_df["adj close"].shift(periods=-n) - acf_df["adj close"]

In [19]:
# Price change n days into the future
n = 1

acf_df[f"delta {n}"] = acf_df["adj close"].shift(periods=-n) - acf_df["adj close"]

acf_plot2 = ts_decompose(ts_df=acf_df[["date", f"delta {n}"]], min_year=min_year, max_year=max_year,
                         lags=acf_lags, pacf_lag=10)

acf_plot2.write_image("./img/EDA2_ACF2.svg", width=1000, height=800)

![ACF](../img/EDA2_ACF2.svg)

Need to normalize (variance increases as underlying ticker price also increases)

In [20]:
acf_df[f"delta {n} norm"] = acf_df[f"delta {n}"] / acf_df["adj close"].shift(periods=n)

acf_plot3 = ts_decompose(ts_df=acf_df[["date", f"delta {n} norm"]], min_year=min_year, max_year=max_year,
                         lags=acf_lags, pacf_lag=10)

acf_plot3.write_image("./img/EDA2_ACF3.svg", width=1000, height=800)

![ACF](../img/EDA2_ACF3.svg)

This residual is stationary, and has expected characteristics (greater volatility in 2008 and beginning of 2020).

With this detrending method, we calculate the stationary dependent variable for various lags

In [14]:
acf_plot3.show(renderer="browser")

## Calculate Target (*Y*) Variable

In [4]:
my_lags = [1, 2, 5, 10, 15, 20, 40, 60, 120, 180, 252]
min_year = 2005
max_year = 2021

Y_LAGS_STAT = date_close_df.copy()

for n in my_lags:
    Y_LAGS_STAT[str(n)] = (Y_LAGS_STAT["adj close"].shift(periods=-n) - Y_LAGS_STAT["adj close"]) / Y_LAGS_STAT[
        "adj close"].shift(periods=n)

Y_LAGS_STAT = Y_LAGS_STAT[(pd.to_datetime(Y_LAGS_STAT["date"]).dt.year >= min_year) &
                          (pd.to_datetime(Y_LAGS_STAT["date"]).dt.year <= max_year)]

Y_LAGS_STAT.drop(columns="adj close", inplace=True)


Use Dickey-Fuller test to examine stationarity at different lags

In [17]:
adfuller_df = pd.DataFrame(index=["ADF Stat.", "p-value",
                                  "critical value 1%", "critical value 5%", "critical value 10%"])

for n in my_lags:
    temp_adfuller = adfuller(Y_LAGS_STAT[str(n)].dropna(), regression="c", autolag="AIC")

    adfuller_df[f"lag {n}"] = [round(temp_adfuller[0], 5),
                               round(temp_adfuller[1], 5),
                               round(temp_adfuller[4]["1%"], 5),
                               round(temp_adfuller[4]["5%"], 5),
                               round(temp_adfuller[4]["10%"], 5)]

adfuller_df

,lag 1,lag 2,lag 5,lag 10,lag 15,lag 20,lag 40,lag 60,lag 120,lag 180,lag 252
ADF Stat.,-15.36720,-10.85631,-9.90605,-9.12094,-8.17547,-9.01673,-9.22906,-6.56879,-4.88917,-4.18588,-3.89412
p-value,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00004,0.00070,0.00208
critical value 1%,-3.43188,-3.43189,-3.43189,-3.43189,-3.43189,-3.43189,-3.43190,-3.43190,-3.43193,-3.43195,-3.43198
critical value 5%,-2.86222,-2.86222,-2.86222,-2.86222,-2.86222,-2.86222,-2.86222,-2.86223,-2.86224,-2.86225,-2.86226
critical value 10%,-2.56713,-2.56713,-2.56713,-2.56713,-2.56713,-2.56713,-2.56713,-2.56714,-2.56714,-2.56715,-2.56715


#### The further away from lag = 1, the less the dependent variable (lag n) remains stationary. Individual years may be non-stationary

In [5]:
test_lag = "1"
dickey_df = pd.DataFrame(columns=["year", "ADF statistic", "p-value"])

for n in range(min_year, max_year + 1):
    # Filter by year
    year_df = Y_LAGS_STAT[pd.to_datetime(Y_LAGS_STAT["date"]).dt.year == n]
    # Get stats
    if not year_df.empty:
        adf_year = adfuller(year_df[test_lag].dropna(), regression="c", autolag="AIC")
        dickey_df = dickey_df.append({"year": n, "ADF statistic": adf_year[0], "p-value": adf_year[1]},
                                     ignore_index=True)

dickey_df["year"] = dickey_df["year"].astype(int)

dickey_df

,year,ADF statistic,p-value
0,2005,-15.738123,1.265472e-28
1,2006,-15.627311,1.702459e-28
2,2007,-12.222780,1.097625e-22
3,2008,-12.570767,2.006525e-23
4,2009,-11.630881,2.264155e-21
5,2010,-15.381526,3.402503e-28
6,2011,-9.293361,1.163628e-15
7,2012,-15.668640,1.522426e-28
8,2013,-6.142854,7.894461e-08
9,2014,-15.365245,3.568136e-28


In [13]:
acf_plot4 = ts_decompose(ts_df=Y_LAGS_STAT[["date", test_lag]], min_year=min_year, max_year=max_year,
                         lags=list(np.arange(1, 252)), pacf_lag=10)

acf_plot4.write_image("./img/EDA2_ACF4.svg", width=1000, height=800)

In [14]:
acf_plot4.show(renderer="browser")

![ACF](../img/EDA2_ACF4.svg)

## Fit Linear Models

Fit linear regression based on various features
1. Baseline linear regression between (days until expiry (DTE) vs. moneyness)
2. DTE vs. moneyness * (+/- 1) depending on sign of delta interest / volume
3. DTE vs. moneyness * (+/- 1) weighted by delta interest / volume
4. DTE vs. moneyness * (+/- 1) weighted by adj. ask price
5. DTE vs. moneyness * (+/- 1) weighted by delta interest / volume * ask price

**The slope and intercept of these fits are used to characterize daily option "sentiment".**

In [6]:
start_time = time.time()
my_pool = Pool(multiprocessing.cpu_count())

# Bookkeeping variables
model_types = ["baseline", "sign", "tag", "price", "er"]
LR_RESULTS = {"delta interest": {"call": {}, "put": {}}, "volume": {"call": {}, "put": {}}}
input_list = []

# Create input list to be processed
for year in list(COMPLETE_DICT.keys()):
    year_df = COMPLETE_DICT[year]
    year_dates = np.unique(year_df["date"])
    for date in year_dates:
        date_df = year_df[year_df["date"] == date]
        for n_1 in ["delta interest", "volume"]:
            for n_2 in ["call", "put"]:
                input_list.append({"year_df": COMPLETE_DICT[year], "date": date,
                                   "n_1": n_1, "n_2": n_2})

results = my_pool.map(fit_linear_model, input_list)

for n in results:
    # Print output messages
    for message in n["messages"]:
        print(message)
    # Unpack results to desired format
    n_1 = n["n_1"]
    n_2 = n["n_2"]
    for model in model_types:
        temp_list = [n["date"]]
        temp_list.extend(n[model])
        if model not in LR_RESULTS[n_1][n_2].keys():
            LR_RESULTS[n_1][n_2][model] = [temp_list]
        else:
            LR_RESULTS[n_1][n_2][model].append(temp_list)

# Convert lists to DataFrames
for n_1 in ["delta interest", "volume"]:
    for n_2 in ["call", "put"]:
        for model in model_types:
            LR_RESULTS[n_1][n_2][model] = pd.DataFrame(LR_RESULTS[n_1][n_2][model],
                                                       columns=["date", "coef", "intercept"])

num_fits = len(input_list) * len(model_types)
print(f"Fitting approx. {num_fits} models took {round(time.time() - start_time, 3)} seconds!")

2006-05-22: No call options with delta interest != 0
2006-05-22: No put options with delta interest != 0
2006-05-25: No call options with delta interest != 0
2006-05-25: No put options with delta interest != 0
2006-10-20: No call options with delta interest != 0
2006-10-20: No put options with delta interest != 0
2008-06-26: No call options with delta interest != 0
2008-06-26: No put options with delta interest != 0
2008-09-19: No call options with delta interest != 0
Fitting approx. 85440 models took 113.784 seconds!


### Kendall rank correlation of: linear slope / intercept coefficient vs. Y

In [24]:
from scipy.stats import kendalltau

TAU_RESULTS = {"delta interest": {"call": {}, "put": {}}, "volume": {"call": {}, "put": {}}}

for n_1 in ["delta interest", "volume"]:
    for n_2 in ["call", "put"]:
        for n_3 in model_types:
            TAU_RESULTS[n_1][n_2][n_3] = dict()
            my_years = np.arange(min_year, max_year + 1)
            TAU_RESULTS[n_1][n_2][n_3]["coef"] = pd.DataFrame(index=my_years, columns=my_lags)
            TAU_RESULTS[n_1][n_2][n_3]["intercept"] = pd.DataFrame(index=my_years, columns=my_lags)

            temp_df1 = LR_RESULTS[n_1][n_2][n_3]
            temp_df1 = temp_df1.merge(Y_LAGS_STAT, how="left", on="date", validate="1:1")
            temp_df1 = temp_df1

            for n_4 in my_years:
                temp_df2 = temp_df1[pd.to_datetime(temp_df1["date"]).dt.year == n_4]
                for n_5 in my_lags:
                    temp_df3 = temp_df2[["coef", "intercept", str(n_5)]].dropna()
                    if not temp_df3.empty:
                        [c_tau, c_pval] = kendalltau(temp_df3["coef"], temp_df3[str(n_5)])
                        [i_tau, i_pval] = kendalltau(temp_df3["intercept"], temp_df3[str(n_5)])

                        TAU_RESULTS[n_1][n_2][n_3]["coef"].loc[n_4, n_5] = round(c_tau, 6)
                        TAU_RESULTS[n_1][n_2][n_3]["intercept"].loc[n_4, n_5] = round(i_tau, 6)


In [50]:
import plotly.graph_objects as go

xaxis=TAU_RESULTS["delta interest"]["call"]["baseline"]["coef"].columns
yaxis=TAU_RESULTS["delta interest"]["call"]["baseline"]["coef"].index


fig = go.Figure(data=[
    go.Surface(
        x=xaxis,
        y=yaxis,
        z=np.abs(TAU_RESULTS["delta interest"]["call"]["baseline"]["coef"]), opacity=0.2
    ),
    # go.Surface(
    #     x=xaxis,
    #     y=yaxis,
    #     z=np.abs(TAU_RESULTS["delta interest"]["call"]["baseline"]["intercept"]), opacity=0.2
    # ),
    # go.Surface(
    #     x=xaxis,
    #     y=yaxis,
    #     z=np.abs(TAU_RESULTS["delta interest"]["put"]["baseline"]["coef"]), opacity=0.2
    # ),
    # go.Surface(
    #     x=xaxis,
    #     y=yaxis,
    #     z=np.abs(TAU_RESULTS["delta interest"]["put"]["baseline"]["intercept"]), opacity=0.2
    # )
])

fig.show(renderer="browser")

In [35]:
TAU_RESULTS["delta interest"]["call"]["baseline"]["coef"]

,1,2,5,10,15,20,40,60,120,180,252
2005,-0.013912,-0.033574,0.046554,0.091663,0.119357,0.095454,0.131245,0.05041,-0.252819,-0.310265,-0.206747
2006,0.005942,-0.042314,-0.095925,-0.186104,-0.209612,-0.122176,-0.114209,-0.145031,-0.176897,-0.09253,-0.164033
2007,-0.049753,-0.070725,-0.094438,-0.133578,-0.096542,-0.070661,0.01259,0.14894,0.455426,0.400096,0.356112
2008,-0.052813,-0.067155,-0.081179,-0.050072,-0.021068,-0.004175,0.175267,0.323028,0.422024,0.221992,0.08494
2009,-0.05369,-0.039461,-0.059318,-0.110921,-0.144375,-0.216278,-0.327895,-0.386707,-0.178334,0.155442,-0.016063
2010,-0.03826,-0.031493,-0.053943,-0.071334,-0.08297,-0.111364,-0.1273,-0.132676,-0.056915,0.091381,0.061911
2011,0.069942,0.04667,0.016758,-0.105799,-0.173655,-0.171378,-0.245494,-0.190729,-0.120154,-0.109024,0.113894
2012,0.043855,0.03306,0.030747,0.067373,0.071871,0.107984,0.071357,0.070779,0.099502,-0.060434,-0.252691
2013,-0.021312,-0.035287,-0.034655,-0.012332,-0.003605,0.002403,-0.014482,-0.061595,0.000253,0.329349,0.243597
2014,0.060462,0.06588,-0.008382,-0.019793,-0.063203,-0.09839,-0.209625,-0.092972,-0.210709,-0.324367,-0.368797


# ----------------------------

**Splitting Data Into Training and Testing**

We split our call and put data into 80% training and 20% testing. No shuffling because data is time-series (order dependent).

In [5]:
all_dates = np.sort(calls_df["date"].unique())

dates_train, dates_test = train_test_split(all_dates, test_size=0.2, random_state=None, shuffle=False)

adj_closing_train = adj_closing_df[adj_closing_df["date"].isin(dates_train)]

### Kendall rank correlations for each type of slope / intercept coefficient

In [14]:
input_list = []
lr_cols = list(LR_fits.columns)
lr_cols.remove("date")
lr_cols.remove("option type")
delta_cols = list(Y_TRAIN_STAT.columns)
delta_cols.remove("date")

for option_type in ["call", "put"]:
    for col1 in lr_cols:
        temp_lr = LR_fits[LR_fits["option type"] == option_type][["date", col1]]
        for col2 in delta_cols:
            temp_delta = Y_TRAIN_STAT[["date", col2]].dropna()
            temp_joined = temp_lr.merge(temp_delta, how="inner", on="date")
            temp_joined.drop(columns="date", inplace=True)
            input_list.append([temp_joined, option_type])


In [15]:
my_pool = Pool(multiprocessing.cpu_count())

results = my_pool.map(kendall_rank, input_list)

# Make sure column names correspond to the order returned by the function
RESULTS_DF = pd.DataFrame(data=results, columns=["tau", "pval", "id1", "id2", "option type"])

### Violin plot to visualize the efficacy of different metrics (probs delete)

In [16]:
tau_violin = go.Figure()

tau_violin.add_trace(go.Violin(x=RESULTS_DF["id1"][RESULTS_DF["option type"] == "call"],
                               y=RESULTS_DF["tau"][RESULTS_DF["option type"] == "call"],
                               name="call",
                               side="positive", line={"color": "orange"}))

tau_violin.add_trace(go.Violin(x=RESULTS_DF["id1"][RESULTS_DF["option type"] == "put"],
                               y=RESULTS_DF["tau"][RESULTS_DF["option type"] == "put"],
                               name="put",
                               side="negative", line={"color": "blue"}))

tau_violin.update_traces(meanline_visible=True)
tau_violin.update_layout(violinmode='overlay',
                         title="Kendall Tau Correlation Distributions of Various Metrics",
                         yaxis_title="Kendall Tau Correlation",
                         font={"size": 14})

tau_violin.show(renderer="browser")

tau_violin.write_image("./img/EDA2_tau_violin.svg", width=1200, height=800)


![violin](../img/EDA2_tau_violin.svg)

### Line plot to visualize the effects of increasing lag on correlation

In [17]:
tau_scatter = make_subplots(rows=1, cols=2, shared_yaxes=True,
                            subplot_titles=("Call", "Put"))

for option_type in ["call", "put"]:
    for metric in np.unique(RESULTS_DF["id1"]):
        if option_type == "call":
            ncol = 1
        else:
            ncol = 2

        tau_scatter.add_trace(
            go.Scatter(x=RESULTS_DF[(RESULTS_DF["option type"] == option_type) & (RESULTS_DF["id1"] == metric)]["id2"],
                       y=RESULTS_DF[(RESULTS_DF["option type"] == option_type) & (RESULTS_DF["id1"] == metric)]["tau"],
                       name=f"{option_type}_{metric}", mode="lines+markers"),
            row=1, col=ncol)

tau_scatter.update_layout(title="Kendall Tau Correlation vs. Lag of Various Metrics",
                          yaxis_title="Kendall Tau Correlation",
                          font={"size": 14})

tau_scatter.show(renderer="browser")

tau_scatter.write_image("./img/EDA2_tau_scatter.svg", width=1600, height=800)


![scatter](../img/EDA2_tau_scatter.svg)

From the above, we see that features `baseline_i`, call `sign_i`, and put `baseline_s` have the best performance. Most features on the put side have an average tau of ~0.05.

We also notice that, aside from call `baseline_i`, the 3 other "significant" features get better the further out we are trying to predict. This is counter-intuitive, because we expect the predictive ability of data to decay the further out we go.

A possible explanation for the poor performance of metrics derived from engineered features (e.g. linear regression weighted by `|change in open contracts|`, or the more complicated `|change in open contracts * ask/bid price|`) could be due to the systematic irregularities of how options are traded. Here are some possible explanations:
- Some options are bought to "lock in" gains in an investor's portfolio. For example, if an investor who recently invested in stock `ABC` wants to cash out, but also wants to avoid the short term capital gains tax. He/she can achieve this by selling call options or buying put options. These movements are usually large (since they want to cover all of their holdings), and are not reflective of the short term "sentiment" of stock `ABC`.
- By taking a look at the net change in open interest on various days, I noticed that the values are usually inflated shortly before ex-dividend dates.
    - For stock `CVX` (Chevron Corporation), there were spikes on 04-28, 05-16, 08-16 and 11-15 in 2016. The ex-dividend dates were 02-16, 05-17, 08-17 and 11-16. This happens with stocks that are dividend heavy, as in the case with `CVX`.
    - This could be because investors want to cash out on the dividends, other quant firms trade dividend events ...etc.

### Manual selection of model based on Kendall Tau

In [21]:
# Visualization of selected model
model_option_type = "put"
model_metric = "baseline_i"
model_lag = "delta 33"

MODEL_DF = LR_fits[["date", model_metric]][LR_fits["option type"] == model_option_type]

MODEL_DF = MODEL_DF.merge(Y_TRAIN_STAT[["date", model_lag]].dropna(),
                          on="date", how="inner")

model_scatter_plot = px.scatter(MODEL_DF, x=model_metric, y=model_lag,
                                marginal_x="violin",
                                marginal_y="violin")

model_scatter_plot.update_layout(
    title=f"{model_lag} vs. {model_option_type}_{model_metric}",
    xaxis_title=f"{model_option_type}_{model_metric}",
    yaxis_title=f"{model_lag}",
    font={"size": 16}
)

model_scatter_plot.show(renderer="browser")

### Method to calculate probability distribution of dependent variable given an independent observation

From the scatter plot, we see that neither the dependent nor independent variables have easy parametric characterizations. It would be hard to fit a well-defined probability distribution on it (e.g. bivariate normal distribution).

Instead, I will apply the following to help "parameterize" the data:
1. Center data
2. Detrend the points using linear regression
3. Divide both the dependent and independent residuals by their marginal standard deviations

After removing all the "parametric" descriptors, we use multivariate kernel density (KDE) estimation to characterize the residuals non-parametrically.
- Gaussian kernel using euclidean distance.
- Scott and Silverman rules to find ideal bandwidths for our KDE. We can compare this to built-in cross validation methods.


In [19]:
MODEL_PROB_DF = MODEL_DF.copy()

# Center data
CENTROID = np.array([np.mean(MODEL_PROB_DF[model_metric]), np.mean(MODEL_PROB_DF[model_lag])])
MODEL_PROB_DF[f"{model_metric}_c"] = MODEL_PROB_DF[model_metric] - CENTROID[0]
MODEL_PROB_DF[f"{model_lag}_c"] = MODEL_PROB_DF[model_lag] - CENTROID[1]

# Detrend using linear regression
MODEL_SCATTER = linear_model.LinearRegression(fit_intercept=False).fit(X=MODEL_PROB_DF[[f"{model_metric}"]],
                                                                       y=MODEL_PROB_DF[[f"{model_lag}"]])

MODEL_PROB_DF[f"{model_lag}_c_detrend"] = (MODEL_PROB_DF[f"{model_lag}_c"] -
                                           (float(MODEL_SCATTER.coef_) * MODEL_PROB_DF[f"{model_metric}_c"]))

# Normalize residuals using marginal standard deviations
Y_SD = np.std(MODEL_PROB_DF[f"{model_lag}_c_detrend"])
X_SD = np.std(MODEL_PROB_DF[f"{model_metric}_c"])

MODEL_PROB_DF[f"{model_lag}_c_detrend_norm"] = MODEL_PROB_DF[f"{model_lag}_c_detrend"] / Y_SD
MODEL_PROB_DF[f"{model_metric}_c_norm"] = MODEL_PROB_DF[f"{model_metric}_c"] / X_SD

# Remove later
margin_plot = px.scatter(MODEL_PROB_DF, x=f"{model_metric}_c_norm", y=f"{model_lag}_c_detrend_norm",
                         marginal_x="violin",
                         marginal_y="violin")

margin_plot.show(renderer="browser")


### Fitting kernel density

In the case of 2 dimensional data here, Scott and Silverman bandwidth are the same.

In [39]:
# KDE using Scott / Silverman
ss_bw = [200 ** (-1 / 6), 200 ** (-1 / 6)]

SS_KDE = KDEMultivariate(data=MODEL_PROB_DF[[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"]],
                         var_type="cc", bw=ss_bw)

# Cross validation max likelihood

CVML_KDE = KDEMultivariate(data=MODEL_PROB_DF[[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"]],
                           var_type="cc", bw="cv_ml")

# Cross validation least squares

CVLS_KDE = KDEMultivariate(data=MODEL_PROB_DF[[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"]],
                           var_type="cc", bw="cv_ls")


Given a new observed value of `x`, we can generate a marginal distribution along `y`.

In [52]:
observed_x = -1
step_size = 0.1
y_range = np.arange(-3, 3 + step_size, step_size)

temp_list = []
for n in y_range:
    temp_list.append([observed_x, n])

PREDICTION_PDF = pd.DataFrame(data=temp_list, columns=[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"])

# Create predictions
PREDICTION_PDF["SS"] = SS_KDE.pdf(PREDICTION_PDF[[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"]])
PREDICTION_PDF["CVML"] = CVML_KDE.pdf(PREDICTION_PDF[[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"]])
PREDICTION_PDF["CVLS"] = CVLS_KDE.pdf(PREDICTION_PDF[[f"{model_metric}_c_norm", f"{model_lag}_c_detrend_norm"]])

In [54]:
# Standardizing the PDF such that the area under is approx. 1



In [53]:
fig = go.Figure()

for my_type in ["SS", "CVML", "CVLS"]:
    fig.add_trace(go.Scatter(x=PREDICTION_PDF[f"{model_lag}_c_detrend_norm"],
                             y=PREDICTION_PDF[my_type],
                             mode='lines+markers', name=my_type))

fig.show(renderer="browser")